In [29]:
tot_sum = open("20_1.txt") do f
    tot = 0

    walkable = Array{Array{Bool,1},1}()
    start = [0,0]
    target = [1,1]
    idx_r = 1
    idx_c = 0 # Making sure we are out of the for scope
    for l in eachline(f)
        idx_c = 1
        push!(walkable, [c !== '#' for c in collect(l)])
        for c in l
            if c == 'S'
                start = [idx_r, idx_c]
            elseif c == 'E'
                target = [idx_r, idx_c]
            end
            idx_c += 1
        end
        idx_r += 1
    end

    # display(walkable)

    nb_rows = idx_r - 1
    nb_cols = idx_c - 1

    dist_from_S = [[nb_rows*nb_rows*nb_cols*nb_cols for j in range(1,nb_cols)] for i in range(1,nb_rows)]
    dist_from_S[start[1]][start[2]] = 0
    dist_from_E = [[nb_rows*nb_rows*nb_cols*nb_cols for j in range(1,nb_cols)] for i in range(1,nb_rows)]
    dist_from_E[target[1]][target[2]] = 0

    # display(dist_from_S)
    # display(dist_from_E)

    delta_neighbours = [[-1,0], [1,0], [0,1], [0,-1]]

    function dijkstra(s, mat)
        frontier = Set([s])
        visited = Set()

        while length(frontier) > 0
            cur_v = nb_rows*nb_rows*nb_cols*nb_cols
            cur_pos = [0,0]
            for pos in frontier
                if mat[pos[1]][pos[2]] < cur_v
                    cur_pos = pos
                    cur_v = mat[pos[1]][pos[2]]
                end
            end
            # We have found our current node 
            # Let's now check the surroundings
            for delta in delta_neighbours
                new_pos = cur_pos + delta

                # TODO: Check if point is walkable AND within reach
                
                if !(new_pos in visited) && walkable[new_pos[1]][new_pos[2]]
                    push!(frontier, new_pos)
                    mat[new_pos[1]][new_pos[2]] = min(mat[new_pos[1]][new_pos[2]], cur_v+1)
                end
            end
            push!(visited, cur_pos)
            delete!(frontier, cur_pos)
        end
        
    end
    dijkstra(start, dist_from_S)
    # display(dist_from_S)
    dijkstra(target, dist_from_E)
    # display(dist_from_E)

    min_delta_ps = 99
    
    # Run through the non walkable and check if we can do better! 
    for i in range(2, nb_cols-1)
        for j in range(2,nb_rows-1)
            available_neighbours = []
            if walkable[i][j]
                continue
            end
            # println("i = ", i, " j = ", j)
            for delta in delta_neighbours
                n = [i,j] + delta
                if walkable[n[1]][n[2]]
                    push!(available_neighbours,n)
                end
            end
            # println("Potential neighbours are ", available_neighbours)
            if length(available_neighbours) > 1
                # Do something
                # println("Let's look into this guy")
                for n1 in available_neighbours
                    for n2 in available_neighbours
                        new_cost = dist_from_S[n1[1]][n1[2]] + dist_from_E[n2[1]][n2[2]]+2
                        if dist_from_S[target[1]][target[2]] - new_cost > min_delta_ps
                            tot += 1
                        end
                    end
                end
                
            end
        end
    end
    (tot)
end

1332

In [57]:
tot_sum = open("20_test.txt") do f
    tot = 0

    walkable = Array{Array{Bool,1},1}()
    start = [0,0]
    target = [1,1]
    idx_r = 1
    idx_c = 0 # Making sure we are out of the for scope
    for l in eachline(f)
        idx_c = 1
        push!(walkable, [c !== '#' for c in collect(l)])
        for c in l
            if c == 'S'
                start = [idx_r, idx_c]
            elseif c == 'E'
                target = [idx_r, idx_c]
            end
            idx_c += 1
        end
        idx_r += 1
    end

    # display(walkable)

    nb_rows = idx_r - 1
    nb_cols = idx_c - 1

    dist_from_S = [[nb_rows*nb_rows*nb_cols*nb_cols for j in range(1,nb_cols)] for i in range(1,nb_rows)]
    dist_from_S[start[1]][start[2]] = 0
    dist_from_E = [[nb_rows*nb_rows*nb_cols*nb_cols for j in range(1,nb_cols)] for i in range(1,nb_rows)]
    dist_from_E[target[1]][target[2]] = 0

    # display(dist_from_S)
    # display(dist_from_E)

    delta_neighbours = [[-1,0], [1,0], [0,1], [0,-1]]

    function dijkstra(s, mat)
        frontier = Set([s])
        visited = Set()

        while length(frontier) > 0
            cur_v = nb_rows*nb_rows*nb_cols*nb_cols
            cur_pos = [0,0]
            for pos in frontier
                if mat[pos[1]][pos[2]] < cur_v
                    cur_pos = pos
                    cur_v = mat[pos[1]][pos[2]]
                end
            end
            # We have found our current node 
            # Let's now check the surroundings
            for delta in delta_neighbours
                new_pos = cur_pos + delta

                # TODO: Check if point is walkable AND within reach
                
                if !(new_pos in visited) && walkable[new_pos[1]][new_pos[2]]
                    push!(frontier, new_pos)
                    mat[new_pos[1]][new_pos[2]] = min(mat[new_pos[1]][new_pos[2]], cur_v+1)
                end
            end
            push!(visited, cur_pos)
            delete!(frontier, cur_pos)
        end
        
    end
    dijkstra(start, dist_from_S)
    # display(dist_from_S)
    dijkstra(target, dist_from_E)
    # display(dist_from_E)

    min_delta_ps = 49
    max_cheat_length = 20
    
    # Run through the non walkable and check if we can do better! 
    cheats = Set()
    for i in range(1, nb_cols)
        for j in range(1,nb_rows)
            if walkable[i][j]
                continue
            end
            println("Looking at blocking element (",i,",",j,")")
            for delta in delta_neighbours
                # Consider all potential neighbours of the starting point
                start_i = i + delta[1]
                start_j = j + delta[2]
                if start_i < 1 || start_j < 1 || start_i > nb_rows || start_j > nb_cols
                    continue
                end
                # println("We're on the grid. ")
                # Let's check if we're walkable for a starting point
                if !walkable[start_i][start_j]
                    continue
                end
                # println("We're on the grid AND on the path")
                println("Considering a starting point (", start_i,",",start_j,").")

                # Review all potential lengths, starting from the first blocking element
                for c_dist in range(1,max_cheat_length-1) # One step has already been taken to get onto the blocking element
                    println("Looking at a cheat of size ", c_dist+1)
                    # Run through all the potential landing element
                    for delta_row in range(-c_dist, c_dist)
                        new_i = delta_row + i
                        if new_i > nb_rows || new_i < 1 
                            continue
                        end
                        left_for_col = c_dist - abs(delta_row)
                        for delta_col in [-left_for_col, left_for_col]
                            new_j = delta_col + j
                            if new_j > nb_cols || new_j < 1
                                continue
                            end
                            # The point is on the grid, we can do something meaningful
                            if !walkable[new_i][new_j]
                                # The cheat does not land where it should
                                continue
                            end
                            # println("The cheat lands on the grid AND is walkable")
                            new_cost = dist_from_S[start_i][start_j] + dist_from_E[new_i][new_j] + c_dist + 1
                            # println("Start cell is (",start_i,",",start_j,"). Landing on (",new_i,",",new_j,"). New_cost is  ", new_cost, " ps" )
                            if dist_from_S[target[1]][target[2]] - new_cost > min_delta_ps
                                if !(([start_i, start_j], [new_i, new_j]) in cheats)
                                    push!(cheats, ([start_i, start_j], [new_i, new_j]))
                                    println("Start cell is (",start_i,",",start_j,"). Landing on (",new_i,",",new_j,"). Saved ", dist_from_S[target[1]][target[2]] - new_cost, " ps" )
                                    tot += 1
                                end
                            end
                        end # Delta col
                    end # delta row
                end # c_dist 
            end # Neighbour of a blocking element
        end # j 
    end #i 

    (tot)
end

Looking at blocking element (1,1)
Looking at blocking element (1,2)
Considering a starting point (2,2).
Looking at a cheat of size 2
Looking at a cheat of size 3
Looking at a cheat of size 4
Looking at a cheat of size 5
Looking at a cheat of size 6
Looking at a cheat of size 7
Looking at a cheat of size 8
Looking at a cheat of size 9
Looking at a cheat of size 10
Start cell is (2,2). Landing on (8,4). Saved 70 ps
Start cell is (2,2). Landing on (10,2). Saved 66 ps
Looking at a cheat of size 11
Start cell is (2,2). Landing on (8,5). Saved 70 ps
Start cell is (2,2). Landing on (9,4). Saved 68 ps
Start cell is (2,2). Landing on (10,3). Saved 66 ps
Start cell is (2,2). Landing on (11,2). Saved 64 ps
Looking at a cheat of size 12
Start cell is (2,2). Landing on (8,6). Saved 70 ps
Start cell is (2,2). Landing on (10,4). Saved 66 ps
Start cell is (2,2). Landing on (12,2). Saved 62 ps
Looking at a cheat of size 13
Start cell is (2,2). Landing on (13,2). Saved 60 ps
Looking at a cheat of size 1

282

In [58]:
# 960864 is too low

In [135]:
tot_sum = open("20_1.txt") do f
    tot = 0

    walkable = Array{Array{Bool,1},1}()
    start = [0,0]
    target = [1,1]
    idx_r = 1
    idx_c = 0 # Making sure we are out of the for scope
    for l in eachline(f)
        idx_c = 1
        push!(walkable, [c !== '#' for c in collect(l)])
        for c in l
            if c == 'S'
                start = [idx_r, idx_c]
            elseif c == 'E'
                target = [idx_r, idx_c]
            end
            idx_c += 1
        end
        idx_r += 1
    end

    # display(walkable)

    nb_rows = idx_r - 1
    nb_cols = idx_c - 1

    dist_from_S = [[nb_rows*nb_rows*nb_cols*nb_cols for j in range(1,nb_cols)] for i in range(1,nb_rows)]
    dist_from_S[start[1]][start[2]] = 0
    dist_from_E = [[nb_rows*nb_rows*nb_cols*nb_cols for j in range(1,nb_cols)] for i in range(1,nb_rows)]
    dist_from_E[target[1]][target[2]] = 0

    # display(dist_from_S)
    # display(dist_from_E)

    delta_neighbours = [[-1,0], [1,0], [0,1], [0,-1]]

    function dijkstra(s, mat)
        frontier = Set([s])
        visited = Set()

        while length(frontier) > 0
            cur_v = nb_rows*nb_rows*nb_cols*nb_cols
            cur_pos = [0,0]
            for pos in frontier
                if mat[pos[1]][pos[2]] < cur_v
                    cur_pos = pos
                    cur_v = mat[pos[1]][pos[2]]
                end
            end
            # We have found our current node 
            # Let's now check the surroundings
            for delta in delta_neighbours
                new_pos = cur_pos + delta

                # TODO: Check if point is walkable AND within reach
                
                if !(new_pos in visited) && walkable[new_pos[1]][new_pos[2]]
                    push!(frontier, new_pos)
                    mat[new_pos[1]][new_pos[2]] = min(mat[new_pos[1]][new_pos[2]], cur_v+1)
                end
            end
            push!(visited, cur_pos)
            delete!(frontier, cur_pos)
        end
        
    end
    dijkstra(start, dist_from_S)
    # display(dist_from_S)
    dijkstra(target, dist_from_E)
    # display(dist_from_E)

    min_delta_ps = 100
    max_cheat_length = 20
    
    # Run through the non walkable and check if we can do better! 
    no_cheat_dist = dist_from_S[target[1]][target[2]]
    cheats = Dict()
    for i in range(1, nb_cols)
        for j in range(1,nb_rows)
            if (!(walkable[i][j])) || (dist_from_S[i][j]+min_delta_ps > no_cheat_dist)
                continue
            end
            # println("Looking at walkable element (",i,",",j,")")
            for delta in delta_neighbours
                # Consider all potential neighbours of the starting point
                start_i = i + delta[1]
                start_j = j + delta[2]
                if start_i < 1 || start_j < 1 || start_i > nb_rows || start_j > nb_cols
                    continue
                end
                # println("We're on the grid. ")
                if walkable[start_i][start_j]
                    continue
                end
                # println("We're on the grid AND on the path")
                # println("Considering a blocking point (", start_i,",",start_j,").")

                # Review all potential lengths, starting from the first blocking element
                for c_dist in range(1,max_cheat_length-1) # One step has already been taken to get onto the blocking element
                    # println("\t\t***\t\t***\t Looking at a cheat of size ", c_dist+1)
                    # Run through all the potential landing element
                    # r_ranges =[]
                    #if delta[1] == 0
                        # r_ranges = -c_dist:c_dist)
                    #else
                    #    r_ranges = [sign(delta[1])*coef for coef in range(0,c_dist)]
                    #end
                    for delta_row in (-c_dist):c_dist
                        new_i = delta_row + start_i
                        # println("Landing i is ", new_i)
                        if new_i > nb_rows || new_i < 1 
                            continue
                        end
                        left_for_col = c_dist - abs(delta_row)
                        # println("Current dist is ", c_dist+1, " delta row from block is ", abs(delta_row), " left_for_col is ", left_for_col)
                        # c_ranges = []
                        #if delta[2] == 0
                            # c_ranges = Set([-left_for_col, left_for_col])
                        #else
                        #    c_ranges = [sign(delta[2])*left_for_col]
                        #end
                        
                        for delta_col in unique([-left_for_col, left_for_col])
                            new_j = delta_col + start_j
                            # println("Landing j is ", new_j)
                            if new_j > nb_cols || new_j < 1
                                continue
                            end
                            # The point is on the grid, we can do something meaningful
                            if !walkable[new_i][new_j]
                                # The cheat does not land where it should
                                continue
                            end
                            # println("The cheat lands on the grid AND is walkable")
                            new_cost = dist_from_S[i][j] + dist_from_E[new_i][new_j] + c_dist + 1
                            gain = no_cheat_dist - new_cost
                            println("Delta course is ", gain, ": (",i,",",j,") => (",start_i, ",", start_j, ") => (",new_i,",",new_j,"). New_cost  = ", new_cost, ". C length = ", c_dist+1, " \t\t" )

                            if gain >= min_delta_ps
                                # if !(([i, j], [new_i, new_j]) in cheats)
                                #    push!(cheats, ([i, j], [new_i, new_j]))
                                #    println("Start cell is (",i,",",j,"). Landing on (",new_i,",",new_j,"). Cheat length = ", c_dist+1, " Saved ", dist_from_S[target[1]][target[2]] - new_cost, " ps" )
                                # println("FOUND NICE PATH: Start cell is (",i,",",j,"). Landing on (",new_i,",",new_j,"). Cheat length = ", c_dist+1, " Saved ", dist_from_S[target[1]][target[2]] - new_cost, " ps" )
                                if ([i, j], [new_i, new_j]) in keys(cheats)
                                    println("Updating course. Previous value was ", cheats[([i, j], [new_i, new_j])])
                                    cheats[([i, j], [new_i, new_j])] = max(cheats[([i, j], [new_i, new_j])], gain)
                                    println("New value is ", cheats[([i, j], [new_i, new_j])], ". tot is ", tot )
                                else
                                    cheats[([i, j], [new_i, new_j])] = gain
                                    tot += 1
                                    println("New cheat found. Now ", tot, " found")
                                end
                            end
                        end # Delta col
                    end # delta row
                end # c_dist 
            end # Neighbour of a blocking element
        end # j 
    end #i 

    nb_occurrences = [cheats[k] for k in keys(cheats)]
    println([(i, count(==(i), nb_occurrences)) for i in unique(nb_occurrences)])


    (tot)
end

Delta course is -2: (2,2) => (1,2) => (2,2). New_cost  = 9442. C length = 2 		
Delta course is -2: (2,2) => (1,2) => (2,3). New_cost  = 9442. C length = 3 		
Delta course is -4: (2,2) => (1,2) => (3,2). New_cost  = 9444. C length = 3 		
Delta course is -2: (2,2) => (1,2) => (2,4). New_cost  = 9442. C length = 4 		
Delta course is -6: (2,2) => (1,2) => (4,2). New_cost  = 9446. C length = 4 		
Delta course is -2: (2,2) => (1,2) => (3,4). New_cost  = 9442. C length = 5 		
Delta course is -8: (2,2) => (1,2) => (5,2). New_cost  = 9448. C length = 5 		
Delta course is 154: (2,2) => (1,2) => (2,6). New_cost  = 9286. C length = 6 		
New cheat found. Now 1 found
Delta course is -2: (2,2) => (1,2) => (4,4). New_cost  = 9442. C length = 6 		
Delta course is -10: (2,2) => (1,2) => (6,2). New_cost  = 9450. C length = 6 		
Delta course is 154: (2,2) => (1,2) => (2,7). New_cost  = 9286. C length = 7 		
New cheat found. Now 2 found
Delta course is 152: (2,2) => (1,2) => (3,6). New_cost  = 9288. C leng

Excessive output truncated after 524289 bytes.

Delta course is -52: (2,22) => (3,22) => (4,12). New_cost  = 9492. C length = 12 		
Delta course is 52: (2,22) => (3,22) => (4,32). New_cost  = 9388. C length = 12 		
Delta course is -72: (2,22) => (3,22) => (6,14). New_cost  = 9512. C length = 12 		
Delta course is 40: (2,22) => (3,22) => (6,30). New_cost  = 9400. C length = 12 		
Delta course is -76: (2,22) => (3,22) => (8,16). New_cost  = 9516. C length = 12 		
Delta course is 4: (2,22) => (3,22) => (8,28). New_cost  = 9436. C length = 12 		
Delta course is -32: (2,22) => (3,22) => (10,18). New_cost  = 9472. C length = 12 		
Delta course is 12: (2,22) => (3,22) => (10,26). New_cost  = 9428. C length = 12 		
Delta course is -88: (2,22) => (3,22) => (12,20). New_cost  = 9528. C length = 12 		
Delta course is -92: (2,22) => (3,22) => (12,24). New_cost  = 9532. C length = 12 		
Delta course is -96: (2,22) => (3,22) => (14,22). New_cost  = 9536. C length = 12 		
Delta course is -56: (2,22) => (3,22) => (2,11). New_cost  = 9496. C length 

LoadError: InterruptException:

In [102]:
for i in -5:5
    println(i)
end

-5
-4
-3
-2
-1
0
1
2
3
4
5


In [7]:
tot_sum = open("20_1.txt") do f
    tot = 0

    walkable = Array{Array{Bool,1},1}()
    start = [0,0]
    target = [1,1]
    idx_r = 1
    idx_c = 0 # Making sure we are out of the for scope
    for l in eachline(f)
        idx_c = 1
        push!(walkable, [c !== '#' for c in collect(l)])
        for c in l
            if c == 'S'
                start = [idx_r, idx_c]
            elseif c == 'E'
                target = [idx_r, idx_c]
            end
            idx_c += 1
        end
        idx_r += 1
    end

    # display(walkable)

    nb_rows = idx_r - 1
    nb_cols = idx_c - 1

    dist_from_S = [[nb_rows*nb_rows*nb_cols*nb_cols for j in range(1,nb_cols)] for i in range(1,nb_rows)]
    dist_from_S[start[1]][start[2]] = 0
    dist_from_E = [[nb_rows*nb_rows*nb_cols*nb_cols for j in range(1,nb_cols)] for i in range(1,nb_rows)]
    dist_from_E[target[1]][target[2]] = 0

    # display(dist_from_S)
    # display(dist_from_E)

    delta_neighbours = [[-1,0], [1,0], [0,1], [0,-1]]

    function dijkstra(s, mat)
        frontier = Set([s])
        visited = Set()

        while length(frontier) > 0
            cur_v = nb_rows*nb_rows*nb_cols*nb_cols
            cur_pos = [0,0]
            for pos in frontier
                if mat[pos[1]][pos[2]] < cur_v
                    cur_pos = pos
                    cur_v = mat[pos[1]][pos[2]]
                end
            end
            # We have found our current node 
            # Let's now check the surroundings
            for delta in delta_neighbours
                new_pos = cur_pos + delta

                # TODO: Check if point is walkable AND within reach
                
                if !(new_pos in visited) && walkable[new_pos[1]][new_pos[2]]
                    push!(frontier, new_pos)
                    mat[new_pos[1]][new_pos[2]] = min(mat[new_pos[1]][new_pos[2]], cur_v+1)
                end
            end
            push!(visited, cur_pos)
            delete!(frontier, cur_pos)
        end
        
    end
    dijkstra(start, dist_from_S)
    # display(dist_from_S)
    dijkstra(target, dist_from_E)
    # display(dist_from_E)

    min_delta_ps = 100
    max_cheat_length = 20
    
    # Run through the non walkable and check if we can do better! 
    no_cheat_dist = dist_from_S[target[1]][target[2]]
    cheats = Dict()
    for i in range(1, nb_cols)
        for j in range(1,nb_rows)
            if (!(walkable[i][j])) || (dist_from_S[i][j]+min_delta_ps > no_cheat_dist)
                continue
            end
            # println("Looking at walkable element (",i,",",j,")")
            # for delta in delta_neighbours
            #     # Consider all potential neighbours of the starting point
                 start_i = i
                 start_j = j
            #     if start_i < 1 || start_j < 1 || start_i > nb_rows || start_j > nb_cols
            #         continue
            #     end
            #     # println("We're on the grid. ")
            #     if walkable[start_i][start_j]
            #        continue
            #     end
            #    # println("We're on the grid AND on the path")
            #     # println("Considering a blocking point (", start_i,",",start_j,").")

                # Review all potential lengths, starting from the first blocking element
                for c_dist in range(2,max_cheat_length) # One step has already been taken to get onto the blocking element
                    # println("\t\t***\t\t***\t Looking at a cheat of size ", c_dist+1)
                    # Run through all the potential landing element
                    # r_ranges =[]
                    #if delta[1] == 0
                        # r_ranges = -c_dist:c_dist)
                    #else
                    #    r_ranges = [sign(delta[1])*coef for coef in range(0,c_dist)]
                    #end
                    for delta_row in (-c_dist):c_dist
                        new_i = delta_row + start_i
                        # println("Landing i is ", new_i)
                        if new_i > nb_rows || new_i < 1 
                            continue
                        end
                        left_for_col = c_dist - abs(delta_row)
                        # println("Current dist is ", c_dist+1, " delta row from block is ", abs(delta_row), " left_for_col is ", left_for_col)
                        # c_ranges = []
                        #if delta[2] == 0
                            # c_ranges = Set([-left_for_col, left_for_col])
                        #else
                        #    c_ranges = [sign(delta[2])*left_for_col]
                        #end
                        
                        for delta_col in unique([-left_for_col, left_for_col])
                            new_j = delta_col + start_j
                            # println("Landing j is ", new_j)
                            if new_j > nb_cols || new_j < 1
                                continue
                            end
                            # The point is on the grid, we can do something meaningful
                            if !walkable[new_i][new_j]
                                # The cheat does not land where it should
                                continue
                            end
                            # println("The cheat lands on the grid AND is walkable")
                            new_cost = dist_from_S[i][j] + dist_from_E[new_i][new_j] + c_dist
                            gain = no_cheat_dist - new_cost
                            # println("Delta course is ", gain, ": (",i,",",j,") => (",start_i, ",", start_j, ") => (",new_i,",",new_j,"). New_cost  = ", new_cost, ". C length = ", c_dist+1, " \t\t" )

                            if gain >= min_delta_ps
                                # if !(([i, j], [new_i, new_j]) in cheats)
                                #    push!(cheats, ([i, j], [new_i, new_j]))
                                #    println("Start cell is (",i,",",j,"). Landing on (",new_i,",",new_j,"). Cheat length = ", c_dist+1, " Saved ", dist_from_S[target[1]][target[2]] - new_cost, " ps" )
                                # println("FOUND NICE PATH: Start cell is (",i,",",j,"). Landing on (",new_i,",",new_j,"). Cheat length = ", c_dist+1, " Saved ", dist_from_S[target[1]][target[2]] - new_cost, " ps" )
                                if ([i, j], [new_i, new_j]) in keys(cheats)
                                    # println("Updating course. Previous value was ", cheats[([i, j], [new_i, new_j])])
                                    cheats[([i, j], [new_i, new_j])] = max(cheats[([i, j], [new_i, new_j])], gain)
                                    # println("New value is ", cheats[([i, j], [new_i, new_j])], ". tot is ", tot )
                                else
                                    cheats[([i, j], [new_i, new_j])] = gain
                                    tot += 1
                                    # println("New cheat found. Now ", tot, " found")
                                end
                            end
                        end # Delta col
                    end # delta row
                end # c_dist 
            # end # Neighbour of a blocking element
        end # j 
    end #i 
    


    nb_occurrences = [cheats[k] for k in keys(cheats)]
    println([(i, count(==(i), nb_occurrences)) for i in unique(nb_occurrences)])


    (tot)
end

[(720, 368), (544, 2378), (338, 2066), (360, 3744), (7690, 13), (656, 1282), (402, 1807), (268, 4273), (102, 5481), (176, 6150), (282, 2526), (522, 1258), (164, 6124), (126, 4528), (174, 3555), (116, 8894), (328, 3941), (280, 4250), (528, 2213), (220, 4671), (216, 5152), (316, 5126), (148, 7154), (452, 2367), (194, 3021), (336, 3649), (356, 3406), (256, 4506), (642, 610), (9092, 348), (304, 4130), (384, 3296), (440, 2615), (114, 5022), (158, 3758), (750, 125), (926, 210), (202, 2977), (538, 1209), (312, 4406), (388, 3183), (1004, 270), (4708, 59), (3956, 117), (654, 656), (184, 5487), (204, 5686), (140, 6948), (324, 4468), (586, 923), (518, 1305), (558, 1303), (632, 1174), (332, 4015), (2292, 169), (682, 334), (146, 4015), (112, 8649), (232, 4530), (1020, 341), (124, 7835), (242, 2630), (226, 2713), (108, 9637), (294, 2548), (248, 4833), (648, 1117), (306, 2491), (1276, 117), (110, 5149), (462, 1257), (422, 1702), (208, 4912), (426, 1567), (270, 2572), (382, 1822), (1932, 78), (578, 97

987695